## Decomposed Fan Utility Model for η and F

This script refines the latent fan-utility index \( \eta_{i,s,t} \) from Question 1 by decomposing it into:

\[
\eta_{i,s,t} \approx \underbrace{\eta^{\text{lin}}_{i,s,t}}_{\text{linear backbone}}
\;+\;
\underbrace{\eta^{\text{ML}}_{i,s,t}}_{\text{nonlinear attribute correction}}.
\]

**Inputs**

- `2026_MCM_Problem_C_Data.csv` (COMAP raw wide-format data)
- `fan_shares_estimated.csv` (Question 1 output), with at least:
  - `season`, `week`, `celebrity_name`
  - `fan_share_hat` (reconstructed fan share)
  - `judge_std` (standardized judge score)

**Modeling steps**

1. **Construct panel with attributes.**
   - Merge `fan_shares_estimated.csv` with static contestant attributes from the raw COMAP file:
     - `celebrity_age_during_season`
     - `celebrity_industry`
     - `ballroom_partner`
     - `celebrity_homecountry/region`.
   - Define the baseline log-utility:
     \[
     \eta_{\text{hat}} = \log(\text{fan\_share\_hat} + 10^{-12}).
     \]

2. **Linear backbone for \(\eta\).**
   - Fit a linear regression:
     \[
     \eta_{\text{hat}} \sim \text{judge\_std} + \text{week}
     + \text{age} + \text{industry} + \text{pro} + \text{homecountry},
     \]
     where numeric features are standardized and categorical features are one-hot encoded.
   - Store residuals \(\eta_{\text{resid}} = \eta_{\text{hat}} - \eta^{\text{lin}}\).

3. **GBDT for residual structure.**
   - Fit a `GradientBoostingRegressor` on \(\eta_{\text{resid}}\) using *only static attributes*:
     - age, industry, pro partner, home country.
   - This learns a flexible nonlinear correction \(\eta^{\text{ML}}\) capturing interaction effects and attribute patterns not explained by the linear model.

4. **Reconstructed decomposed utility and fan shares.**
   - For every panel row, compute:
     - `eta_lin` — linear prediction from the backbone.
     - `eta_ml` — nonlinear correction from GBDT.
     - `eta_decomp = eta_lin + eta_ml`.
   - Convert `eta_decomp` to implied fan shares via a **within-week softmax**:
     \[
     F^{\text{decomp}}_{i,s,t}
       = \frac{\exp(\eta^{\text{decomp}}_{i,s,t})}
              {\sum_{j \in \mathcal{A}_{s,t}} \exp(\eta^{\text{decomp}}_{j,s,t})},
     \]
     saved as column `F_decomp`.

**Outputs**

- `panel_with_eta_decomp.csv`:
  - original panel plus:
    - `eta_hat`, `eta_resid`
    - `eta_lin`, `eta_ml`, `eta_decomp`
    - `F_decomp`.
- Serialized models for reuse and diagnostics:
  - `lin_eta_model.joblib` and `lin_feature_names.joblib`
  - `gbdt_eta_model.joblib` and `gbdt_feature_names.joblib`.

These objects are used later in the report to:
- interpret linear coefficients (e.g., effect of age or industry on \(\eta\)),
- inspect GBDT feature importances for nonlinear attribute effects,
- and plug `F_decomp` into downstream simulations of alternative voting rules.


In [ ]:
"""
eta_decomposition.py

Refine the latent utility index eta_hat for DWTS fan support by:
  - Fitting a linear backbone model in judge_std, week, and static attributes.
  - Fitting a Gradient Boosting model on the residuals using static attributes.
  - Reconstructing a decomposed utility:
        eta_decomp = eta_lin + eta_ml
    and the implied fan shares via softmax within each (season, week).

Inputs:
  - 2026_MCM_Problem_C_Data.csv       (raw COMAP-style wide data)
  - fan_shares_estimated.csv          (Question 1 output)

Outputs:
  - panel_with_eta_decomp.csv         (long panel with eta/F decomposition)
  - lin_eta_model.joblib              (linear backbone model)
  - lin_feature_names.joblib          (names of encoded linear features)
  - gbdt_eta_model.joblib             (GBDT residual model)
  - gbdt_feature_names.joblib         (names of encoded GBDT features)
"""

from __future__ import annotations

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from joblib import dump


# ============================================================
# 0. File paths (change these if needed)
# ============================================================

RAW_DATA_PATH = "2026_MCM_Problem_C_Data.csv"
FAN_EST_PATH = "fan_shares_estimated.csv"


# ============================================================
# 1. Helper: numerically stable softmax
# ============================================================

def softmax_stable(x: np.ndarray) -> np.ndarray:
    """
    Numerically stable softmax for a 1D numpy array.
    Used within each (season, week) group to convert eta -> F.
    """
    x = np.asarray(x)
    x_shifted = x - x.max()
    ex = np.exp(x_shifted)
    return ex / ex.sum()


# ============================================================
# 2. Linear backbone for eta_hat
# ============================================================

def fit_eta_linear(df_panel: pd.DataFrame):
    """
    Fit a linear regression for eta_hat as a function of:
      - judge_std, week, age (numeric)
      - industry, pro partner, homecountry (categorical)

    Parameters
    ----------
    df_panel : DataFrame
        Must contain columns:
          - 'eta_hat'
          - 'judge_std'
          - 'week'
          - 'celebrity_age_during_season'
          - 'celebrity_industry'
          - 'ballroom_partner'
          - 'celebrity_homecountry/region'

    Returns
    -------
    lin_model : Pipeline
        Preprocessing + LinearRegression model for eta_hat.
    df_out : DataFrame
        Copy of df_panel with an extra column 'eta_resid' (residuals).
    feature_names : np.ndarray
        Names of the encoded features used in the linear model.
    """
    df = df_panel.copy()
    y = df["eta_hat"].values

    # Numeric features in the linear part
    num_features = ["judge_std", "week", "celebrity_age_during_season"]

    # Categorical features – one-hot encoded
    home_country_col = "celebrity_homecountry/region"
    cat_features = [
        "celebrity_industry",
        "ballroom_partner",
        home_country_col,
    ]

    X = df[num_features + cat_features]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_features),
            ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), cat_features),
        ]
    )

    lin_model = Pipeline(steps=[
        ("prep", preprocessor),
        ("lin", LinearRegression())
    ])

    lin_model.fit(X, y)
    y_pred_lin = lin_model.predict(X)
    residuals = y - y_pred_lin

    print("Linear model R^2 on eta_hat: {:.3f}".format(r2_score(y, y_pred_lin)))

    df["eta_resid"] = residuals

    # Recover feature names for interpretation
    ohe = lin_model.named_steps["prep"].named_transformers_["cat"]
    cat_encoded_names = ohe.get_feature_names_out(cat_features)
    feature_names = np.concatenate([num_features, cat_encoded_names])

    # Inspect largest coefficients (optional, but nice to see in logs)
    coefs = lin_model.named_steps["lin"].coef_
    coef_df = (
        pd.DataFrame({"feature": feature_names, "coef_eta": coefs})
        .sort_values("coef_eta", key=lambda s: s.abs(), ascending=False)
    )

    print("\nTop 20 linear coefficients for eta_hat:")
    print(coef_df.head(20))

    return lin_model, df, feature_names


# ============================================================
# 3. GBDT model on eta residuals
# ============================================================

def fit_eta_residual_gbdt(df_with_resid: pd.DataFrame):
    """
    Fit GradientBoostingRegressor on residuals of eta_hat
    using static attributes only (no judge_std or week).

    Parameters
    ----------
    df_with_resid : DataFrame
        Must contain 'eta_resid' and static attribute columns.

    Returns
    -------
    gbdt_model : Pipeline
        Preprocessing + GradientBoostingRegressor trained on residuals.
    feat_names : np.ndarray
        Encoded feature names used by the GBDT.
    """
    df = df_with_resid.copy()
    r = df["eta_resid"].values

    # Only static attributes here
    num_features = ["celebrity_age_during_season"]
    home_country_col = "celebrity_homecountry/region"
    cat_features = [
        "celebrity_industry",
        "ballroom_partner",
        home_country_col,
    ]

    X = df[num_features + cat_features]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_features),
            ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), cat_features),
        ]
    )

    gbdt_model = Pipeline(steps=[
        ("prep", preprocessor),
        ("gbr", GradientBoostingRegressor(
            random_state=0,
            n_estimators=300,
            max_depth=3,
            learning_rate=0.05,
            subsample=0.8,
        )),
    ])

    gbdt_model.fit(X, r)
    r_pred = gbdt_model.predict(X)

    print("GBDT R^2 on eta residuals: {:.3f}".format(r2_score(r, r_pred)))

    ohe = gbdt_model.named_steps["prep"].named_transformers_["cat"]
    cat_names = ohe.get_feature_names_out(cat_features)
    feat_names = np.concatenate([num_features, cat_names])

    importances = gbdt_model.named_steps["gbr"].feature_importances_
    order = np.argsort(importances)[::-1]

    print("\nTop 20 GBDT feature importances for residual eta:")
    for idx in order[:20]:
        print(f"{feat_names[idx]:40s} {importances[idx]:.4f}")

    return gbdt_model, feat_names


# ============================================================
# 4. Build eta_lin, eta_ml, eta_decomp, F_decomp
# ============================================================

def add_eta_and_F_decomp(df_panel: pd.DataFrame,
                         lin_model: Pipeline,
                         gbdt_model: Pipeline) -> pd.DataFrame:
    """
    Compute decomposed eta and implied fan shares from the fitted models.

    Adds the following columns:
      - 'eta_lin'    : linear backbone prediction
      - 'eta_ml'     : nonlinear ML correction (GBDT)
      - 'eta_decomp' : sum of the two
      - 'F_decomp'   : softmax over eta_decomp within each (season, week)
    """
    df = df_panel.copy()

    # Linear part uses judge_std, week, age + categorical attrs
    num_features_all = ["judge_std", "week", "celebrity_age_during_season"]
    home_country_col = "celebrity_homecountry/region"
    cat_features_all = [
        "celebrity_industry",
        "ballroom_partner",
        home_country_col,
    ]
    X_all = df[num_features_all + cat_features_all]
    eta_lin = lin_model.predict(X_all)

    # GBDT part uses static attributes only
    num_features_static = ["celebrity_age_during_season"]
    cat_features_static = [
        "celebrity_industry",
        "ballroom_partner",
        home_country_col,
    ]
    X_static = df[num_features_static + cat_features_static]
    eta_ml = gbdt_model.predict(X_static)

    df["eta_lin"] = eta_lin
    df["eta_ml"] = eta_ml
    df["eta_decomp"] = df["eta_lin"] + df["eta_ml"]

    # Softmax per (season, week) to get decomposed fan shares
    df["F_decomp"] = (
        df.groupby(["season", "week"])["eta_decomp"]
          .transform(softmax_stable)
    )

    return df


# ============================================================
# 5. Main script
# ============================================================

def main() -> None:
    # 1) Load raw COMAP data and fan-share estimates
    df_raw = pd.read_csv(RAW_DATA_PATH)
    df_fan = pd.read_csv(FAN_EST_PATH)

    # Static attributes (one row per contestant-season)
    attrs_cols = [
        "season",
        "celebrity_name",
        "celebrity_age_during_season",
        "celebrity_industry",
        "ballroom_partner",
        "celebrity_homecountry/region",
    ]
    df_attrs = df_raw[attrs_cols].drop_duplicates()

    # Merge attributes into fan panel
    df_panel = df_fan.merge(
        df_attrs,
        on=["season", "celebrity_name"],
        how="left",
    )

    # Define eta_hat as log fan share (Question 1 latent utility proxy)
    df_panel["eta_hat"] = np.log(df_panel["fan_share_hat"] + 1e-12)

    print("df_panel shape:", df_panel.shape)
    print("Columns:", df_panel.columns.tolist())

    # 2) Fit linear backbone
    lin_eta, df_with_resid, lin_feat_names = fit_eta_linear(df_panel)

    # 3) Fit residual GBDT on static attributes
    gbdt_eta, gbdt_feat_names = fit_eta_residual_gbdt(df_with_resid)

    # 4) Build decomposed eta and fan shares
    df_eta_full = add_eta_and_F_decomp(df_with_resid, lin_eta, gbdt_eta)

    print("\nFinished. df_eta_full columns now include:")
    print([c for c in df_eta_full.columns if c.startswith("eta_") or c.startswith("F_")])

    # 5) Save panel and models
    df_eta_full.to_csv("panel_with_eta_decomp.csv", index=False)
    dump(lin_eta, "lin_eta_model.joblib")
    dump(lin_feat_names, "lin_feature_names.joblib")
    dump(gbdt_eta, "gbdt_eta_model.joblib")
    dump(gbdt_feat_names, "gbdt_feature_names.joblib")

    print("Saved panel_with_eta_decomp.csv and model joblib files.")


if __name__ == "__main__":
    main()


## Figure 6: Baseline Fan Popularity by Industry and Professional Partner

This script visualizes how **attribute-driven fan utility** (the nonlinear ML component \(\eta_{\text{ml}}\) from the decomposed model) varies across contestant industries and professional partners.

### Inputs

- `panel_with_eta_decomp.csv`  
  Long panel produced by the eta–decomposition script, containing at least:
  - `season`, `celebrity_name`
  - `eta_ml` – the ML residual utility component
  - (optionally) `celebrity_industry`, `ballroom_partner`

- `2026_MCM_Problem_C_Data.csv`  
  Used only if `celebrity_industry` or `ballroom_partner` are missing from the panel; the script merges those attributes back in.

### Processing Steps

1. **Ensure attributes are present.**  
   If `celebrity_industry` or `ballroom_partner` are missing in `df_eta`, they are merged from the raw COMAP data using `(season, celebrity_name)`.

2. **Collapse to contestant–season level.**  
   For each `(season, celebrity_name)`, the script averages `eta_ml` over weeks:
   \[
   \eta_{\text{ml, mean}} = \text{mean}_t\; \eta_{\text{ml}, i,s,t}
   \]
   The resulting dataframe `df_c` has one row per contestant–season with:
   - `celebrity_industry`
   - `ballroom_partner`
   - `eta_ml_mean`

3. **Industry grouping (Fig. 6a).**
   - Count how many contestants appear in each `celebrity_industry`.
   - Keep the **top 7 industries**, group all others into `"Other"`.
   - For each industry (including `"Other"`), collect the distribution of `eta_ml_mean`.
   - Plot a **horizontal boxplot**:
     - x-axis: `eta_ml_mean` (attribute-driven utility)
     - y-axis: industry categories
     - Title: *“Fig. 6(a). Baseline Fan Popularity by Industry”*

4. **Professional partner grouping (Fig. 6b).**
   - Count how many contestant–seasons each `ballroom_partner` appears in.
   - Keep pros with at least 6 contestants, then take the **top 10** by frequency.
   - For each such partner, collect the distribution of `eta_ml_mean`.
   - Plot another horizontal boxplot:
     - x-axis: `eta_ml_mean`
     - y-axis: professional partners
     - Title: *“Fig. 6(b). Baseline Fan Popularity by Professional Partner”*

5. **Styling and output.**
   - Uses **UMN-style colors**:
     - Maroon outlines (`#7A0019`)
     - Light gold fills (`#FFE6A6`)
   - Removes top/right spines and adds light x-grid lines.
   - Saves the combined figure as:
     - `Fig6_industry_pro.png`
     - `Fig6_industry_pro.pdf`

### Interpretation Use

- Each box summarizes the distribution of **baseline fan popularity** \(\eta_{\text{ml}}\) for a given group.
- Higher medians and upper quartiles indicate industries or professional partners associated with systematically stronger fan support **beyond** what judge scores and week effects explain.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- paths (change if needed) ----------
RAW_DATA_PATH   = "2026_MCM_Problem_C_Data.csv"
PANEL_PATH      = "panel_with_eta_decomp.csv"

# ---------- colors ----------
MAROON     = "#7A0019"
LIGHT_GOLD = "#FFE6A6"

plt.rcParams.update({
    "font.size": 11,
    "axes.titlesize": 13,
    "axes.labelsize": 11,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "figure.facecolor": "white"
})

# =========================================================
# 1. Load panel & make sure attributes exist
# =========================================================
df_eta = pd.read_csv(PANEL_PATH)

needed_cols = ["celebrity_industry", "ballroom_partner"]
missing = [c for c in needed_cols if c not in df_eta.columns]

if missing:
    df_raw = pd.read_csv(RAW_DATA_PATH)
    attrs_cols = ["season", "celebrity_name"] + needed_cols
    df_attrs = df_raw[attrs_cols].drop_duplicates()
    df_eta = df_eta.merge(
        df_attrs,
        on=["season", "celebrity_name"],
        how="left"
    )

# Collapse to contestant–season level
df_c = (
    df_eta
    .groupby(
        ["season", "celebrity_name",
         "celebrity_industry", "ballroom_partner"],
        as_index=False
    )["eta_ml"]
    .mean()
    .rename(columns={"eta_ml": "eta_ml_mean"})
)

# =========================================================
# 2. Build Figure 6
# =========================================================
fig6, axes = plt.subplots(1, 2, figsize=(11, 4.8), sharey=True)

# ---------- Fig. 6(a): Industry ----------
ind_counts = df_c["celebrity_industry"].value_counts()
top_inds = ind_counts.head(7).index  # top 7 industries
df_c["industry_simplified"] = np.where(
    df_c["celebrity_industry"].isin(top_inds),
    df_c["celebrity_industry"],
    "Other"
)

ind_order = (
    df_c.groupby("industry_simplified")["eta_ml_mean"]
        .median()
        .sort_values()
        .index
)

data_ind = [
    df_c.loc[df_c["industry_simplified"] == ind, "eta_ml_mean"]
    for ind in ind_order
]

bp_ind = axes[0].boxplot(
    data_ind,
    vert=False,
    patch_artist=True
)

for patch in bp_ind["boxes"]:
    patch.set(facecolor=LIGHT_GOLD, edgecolor=MAROON, alpha=0.9)
for median in bp_ind["medians"]:
    median.set(color=MAROON, linewidth=2)
for whisker in bp_ind["whiskers"]:
    whisker.set(color=MAROON, linewidth=0.8)
for cap in bp_ind["caps"]:
    cap.set(color=MAROON, linewidth=0.8)
for flier in bp_ind["fliers"]:
    flier.set(marker="o", markersize=3, alpha=0.4,
              markerfacecolor=MAROON, markeredgecolor="none")

axes[0].set_yticks(range(1, len(ind_order) + 1))
axes[0].set_yticklabels(ind_order, fontsize=9)
axes[0].set_xlabel("Attribute-driven utility (η_ml)")
axes[0].set_title("Fig. 6(a). Baseline Fan Popularity by Industry")
axes[0].grid(axis="x", linestyle="--", alpha=0.3)
for spine in ["top", "right"]:
    axes[0].spines[spine].set_visible(False)

# ---------- Fig. 6(b): Professional partner ----------
pro_counts = df_c["ballroom_partner"].value_counts()
top_pros = pro_counts[pro_counts >= 6].head(10).index  # top 10 frequent pros
df_pro = df_c[df_c["ballroom_partner"].isin(top_pros)]

pro_order = (
    df_pro.groupby("ballroom_partner")["eta_ml_mean"]
          .median()
          .sort_values()
          .index
)

data_pro = [
    df_pro.loc[df_pro["ballroom_partner"] == pro, "eta_ml_mean"]
    for pro in pro_order
]

bp_pro = axes[1].boxplot(
    data_pro,
    vert=False,
    patch_artist=True
)

for patch in bp_pro["boxes"]:
    patch.set(facecolor=LIGHT_GOLD, edgecolor=MAROON, alpha=0.9)
for median in bp_pro["medians"]:
    median.set(color=MAROON, linewidth=2)
for whisker in bp_pro["whiskers"]:
    whisker.set(color=MAROON, linewidth=0.8)
for cap in bp_pro["caps"]:
    cap.set(color=MAROON, linewidth=0.8)
for flier in bp_pro["fliers"]:
    flier.set(marker="o", markersize=3, alpha=0.4,
              markerfacecolor=MAROON, markeredgecolor="none")

axes[1].set_yticks(range(1, len(pro_order) + 1))
axes[1].set_yticklabels(pro_order, fontsize=9)
axes[1].set_xlabel("Attribute-driven utility (η_ml)")
axes[1].set_title("Fig. 6(b). Baseline Fan Popularity by Professional Partner")
axes[1].grid(axis="x", linestyle="--", alpha=0.3)
for spine in ["top", "right"]:
    axes[1].spines[spine].set_visible(False)

fig6.tight_layout()

# ---------- Show and save ----------
plt.show()

fig6.savefig("Fig6_industry_pro.png", dpi=300, bbox_inches="tight")
fig6.savefig("Fig6_industry_pro.pdf", dpi=300, bbox_inches="tight")

print("Saved Figure 6 as Fig6_industry_pro.png and Fig6_industry_pro.pdf")


More Plots:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------------
# 1. Average nonlinear fan effect by home country
# ------------------------------------------------

# Average eta_ml by home country
country_effect = (
    df_eta_full
    .groupby("celebrity_homecountry/region")["eta_ml"]
    .mean()
    .sort_values()
)

# Optionally filter to countries with enough data
min_n = 20
country_counts = df_eta_full["celebrity_homecountry/region"].value_counts()
valid_countries = country_counts[country_counts >= min_n].index
country_effect = country_effect.loc[valid_countries]

# Take extremes (e.g., 8 lowest and 8 highest) to keep the plot readable
# country_effect is already sorted
# country_effect: Series indexed by country, values = mean eta_ml

k = min(8, len(country_effect))  # at most 8, but never more than we have

head_part = country_effect.head(k)
tail_part = country_effect.tail(k)

effect_subset = pd.concat([head_part, tail_part])

# Remove any duplicates if the head and tail overlap (e.g., total < 2k)
effect_subset = effect_subset[~effect_subset.index.duplicated(keep="first")]


# Make labels a bit shorter (optional)
effect_subset.index = effect_subset.index.str.replace("United States", "USA")

# ------------------------------------------------
# 2. GBDT feature importances for country dummies
# ------------------------------------------------

importances = gbdt_eta.named_steps["gbr"].feature_importances_
imp_df = pd.DataFrame({
    "feature": gbdt_feat_names,
    "importance": importances,
})

country_imp = (
    imp_df[imp_df["feature"].str.startswith("celebrity_homecountry/region_")]
    .sort_values("importance", ascending=False)
)

# Top 10 most important country dummies
country_imp_top = country_imp.head(10).copy()
# Clean up feature names for display
country_imp_top["label"] = (
    country_imp_top["feature"]
    .str.replace("celebrity_homecountry/region_", "", regex=False)
    .str.replace("United States", "USA")
)

# ------------------------------------------------
# 3. Plot: two subplots side by side
# ------------------------------------------------

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# --- Left: average η_ml effect by country ---
ax = axes[0]
effect_subset.plot(kind="barh", ax=ax)
ax.axvline(0, color="black", linewidth=1)
ax.set_xlabel("Average nonlinear fan utility effect (η_ml)")
ax.set_ylabel("Home country / region")
ax.set_title("Home Country Effect on Fan Support (η_ml)")
ax.grid(axis="x", linestyle="--", alpha=0.4)

# Make labels a bit smaller if many categories
for label in ax.get_yticklabels():
    label.set_fontsize(9)

# --- Right: GBDT importance for country dummies ---
ax2 = axes[1]
ax2.barh(country_imp_top["label"], country_imp_top["importance"])
ax2.invert_yaxis()  # highest importance at top
ax2.set_xlabel("GBDT feature importance")
ax2.set_title("Most Influential Home Country Indicators")
ax2.grid(axis="x", linestyle="--", alpha=0.4)

for label in ax2.get_yticklabels():
    label.set_fontsize(9)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Partial dependence-style plot using eta_ml
age = df_eta_full["celebrity_age_during_season"]
eta_ml = df_eta_full["eta_ml"]

# Bin ages
bins = np.linspace(age.min(), age.max(), 12)
bin_centers = 0.5 * (bins[:-1] + bins[1:])

eta_by_age = []
for lo, hi in zip(bins[:-1], bins[1:]):
    mask = (age >= lo) & (age < hi)
    eta_by_age.append(eta_ml[mask].mean())

plt.figure(figsize=(7, 5))
plt.plot(bin_centers, eta_by_age, marker="o")
plt.axhline(0, color="black", linewidth=1)
plt.xlabel("Celebrity age during season")
plt.ylabel("Average nonlinear fan utility (η_ml)")
plt.title("Nonlinear Effect of Age on Fan Support")
plt.grid(alpha=0.4)
plt.tight_layout()
plt.show()
